# The Battle of Neighborhoods

We import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geocoder --yes
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


import folium # map rendering library

#from bs4 import BeautifulSoup
#import csv

import random


print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.



ModuleNotFoundError: No module named 'geopy'

We get the HTML of the Wiki page, convert into a table with help of read_html (read HTML tables into a list of DataFrame objects), remove cells with a borough that is Not assigned.

In [ ]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wiki)

df_raw = pd.read_html(wikipedia_page.content, header=0)[0]
df_new = df_raw[df_raw.Borough != 'Not assigned']

df_new.head()

We find whether there is a "Not assigned" in Neighbourhood

In [ ]:
df_new.loc[df_new.Neighbourhood == 'Not assigned']

If we have Neighbourhood Not assigned, we change it with the value of Borough

In [ ]:
df_new.Neighbourhood.replace('Not assigned',df_new.Borough,inplace=True)
df_new.head(8)

We group Neighbourhoods with the same Postcode

In [ ]:
df_toronto = df_new.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))
df_toronto = df_toronto.reset_index()
df_toronto.rename(columns = {'Postcode':'PostalCode'}, inplace = True)
df_toronto.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
df_toronto.head()

In [ ]:
df_toronto.shape

 We get the latitude and the longitude coordinates of each neighborhood, using the csv file

In [ ]:
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head()

In [ ]:
#check the sshape o the csv file
df_geo.shape

Both tables have the same chape. Now we can join new colums to our data.

In [ ]:
df_toronto = df_toronto.join(df_geo.set_index('Postal Code'), on='PostalCode')
df_toronto

# Using the Foursquare API

We can use the Foursquare API to get the venues of each neighborhood. But first, let's get the latitude longitude of Toronto.

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We can also print the map of toronto with all neighborhoods marked on it to help visualize

In [ ]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

Those are the possible recommendations spots for our recommendation system.

Before using the Foursquare API, we need to enter the client credentials and secret.

In [ ]:
CLIENT_ID = 'XLBDRGURZVOOULUGDUC4DSJZSRE5ZI0XPB1WA5RV3YL5D1TP' # your Foursquare ID
CLIENT_SECRET = '5TW0I4L1WKC5A0H1NZHEDZD535BFN1CDQ2MRHBW1VBNFUTEK' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

We define a maximum of 100 venues and a 500 meters radius from the center of the neighborhood

In [ ]:
LIMIT = 100
radius = 500

Define a fundction to extract the nearby venues from the json that returns from the API request

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

And now we can get the nearby venues for all neighborhoods

In [ ]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Now, lets check how many venues we have in our database

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

Now, let's check how many venues for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

We can see that some of the neighborhoods reached the 100 venue limit we established. But that shouldn't affect so much the prototype

How many categorys can we find in Toronto?

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Now we can onehot encode to prepare the dataset for our recommendation system

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape


In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

Now the dataset is ready for our recommendation system

# Random user

We need to create a list of all avaible categorys

In [ ]:
category_list = list(toronto_grouped)
del category_list[0]

How many categorys will our user have?

In [ ]:
category_amount = random.randint(1,11)

In [ ]:
user_categories = random.sample(category_list, category_amount)

In [ ]:
aux_list = []
aux_list2 = []
for i in range(len(category_list)):
    aux_list2.append(1)
    if category_list[i] in user_categories:
        aux_list.append(1)
    else:
        aux_list.append(0)
user_categories

Create a dataframe for the user where the columns will have the categories name and the values will be alway 1

In [ ]:
user_df = pd.DataFrame.from_records([aux_list], columns=category_list)
user_df.head()

In [ ]:
aux_df = pd.DataFrame(aux_list2)

In [ ]:
user_profile = user_df.transpose().dot(aux_df.transpose()[0])
#user_profile = user_profile.set_index('index')
#user_profile.rename(columns={'index': 'Categories', '0': 'value'}, inplace = True)
#user_profile = user_profile.set_index('Categories')
user_profile

Now we have the user favorite categories. All we have to do, is get the categories table for all neighborhoods and compare them to see which neighborhood is the best fit for the user.

In [ ]:
categories_table = toronto_grouped.set_index(toronto_grouped['Neighborhood'])
categories_table = categories_table.drop('Neighborhood', 1)
categories_table.head()

Now we multipy the matriz cateregories_table and user_profile to obtain the score for each neighborhood

In [ ]:
recommendationTable_df = ((categories_table*user_profile).sum(axis=1))
recommendationTable_df.head()

In [ ]:
recommendationTable_df = pd.DataFrame(recommendationTable_df.sort_values(ascending=False))

In [ ]:
recommendationTable_df.columns = ['Score']
recommendationTableFinal_df = recommendationTable_df.reset_index()
recommendationTableFinal_df.head()

Now that we have the score for the top neighborhoods, we can join the table with the scores with the table with the location

In [ ]:
df_neighborhood_score = df_toronto.join(recommendationTableFinal_df.set_index('Neighborhood'), on ='Neighborhood')
df_neighborhood_score.head()

In [ ]:
df_neighborhood_score.sort_values(by='Score', ascending=False, inplace = True)
df_neighborhood_score.reset_index(drop=True, inplace = True)
df_neighborhood_score.head()

And with that we can finally print on the map the 5 best neighborhoods for our user

In [ ]:
recommendation_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = 5
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, score, index in zip(df_neighborhood_score['Latitude'], df_neighborhood_score['Longitude'], df_neighborhood_score['Neighborhood'], df_neighborhood_score['Score'], range(5)):
    label = folium.Popup(str(poi) + ' Score ' + str(score), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(index)],
        fill=True,
        fill_color=rainbow[int(index)],
        fill_opacity=0.7).add_to(recommendation_map)
       
recommendation_map

From this example, we can see which neighborhoods are the best for our random user, and by clicking on the printed neighborhoods we can see the scores of each one of them. And with that our user can select the one that best fit his interests